# Bindings - Exercises

## CFFI - Exercise

You are convinced that writing the number-crunchy loops of your Pi approximation as a CFFI extension is a good idea. Implement the necessary parts to prove your point.

___Remarks:___ You do not need to write any C code, that is already done for you. Only the 5th and the 7th cells below require some Python code.

In [ ]:
%%writefile bindings/cffi/exercises/shoot.c

# include <math.h>


void shoot(double *x, double *y, int *h, int n) {
    for (int i = 0; i < n; i++) {
            h[i] = (sqrt(x[i]*x[i] + y[i]*y[i]) <= 1.0) ? 1 : 0;
    }
}

In [ ]:
%%writefile bindings/cffi/exercises/count.c

int count(int *h, int n) {
    int hits = 0;
    
    for (int i = 0; i < n; i++) {
        hits += h[i];
    }
    
    return hits;
}

In [ ]:
%%writefile bindings/cffi/exercises/shoot.h

void shoot(double *x, double *y, int *h, int n);

In [ ]:
%%writefile bindings/cffi/exercises/count.h

int count(int *h, int n);

In [ ]:
%%writefile bindings/cffi/exercises/piapprox.py

from cffi import FFI
ffibuilder = FFI()

ffibuilder.cdef(""" """)

ffibuilder.set_source("_pi_approx",
"""

""",
    sources=[', ''],
    libraries=['']) 

if __name__ == "__main__":
    ffibuilder.compile(verbose=True)

In [ ]:
! cd bindings/cffi/exercises/; python3 piapprox.py

In [ ]:
%%writefile bindings/cffi/exercises/main.py

import numpy as np
from _pi_approx import ffi, lib


n = 1_000
x = np.random.rand(n)
y = np.random.rand(n)
h = np.zeros(n)

xptr = None
yptr = None
hptr = None
lib.shoot(xptr, yptr, hptr, n)
pi = 4*lib.count(hptr, n)/n

print(pi)

#### CFFI - Solution Proposal

In [ ]:
%%writefile bindings/cffi/exercises/shoot.c

# include <math.h>


void shoot(double *x, double *y, int *h, int n) {
    for (int i = 0; i < n; i++) {
            h[i] = (sqrt(x[i]*x[i] + y[i]*y[i]) <= 1.0) ? 1 : 0;
    }
}

In [ ]:
%%writefile bindings/cffi/exercises/piapprox.py

from cffi import FFI
ffibuilder = FFI()

ffibuilder.cdef("""void shoot(double *x, double *y, int *h, int n);
                   int count(int *h, int n);""")

ffibuilder.set_source("_pi_approx",
"""
    #include "shoot.h"
    #include "count.h"
""",
    sources=['shoot.c', 'count.c'],
    libraries=['m']) 

if __name__ == "__main__":
    ffibuilder.compile(verbose=True)

In [ ]:
%%writefile bindings/cffi/exercises/count.c

int count(int *h, int n) {
    int hits = 0;
    
    for (int i = 0; i < n; i++) {
        hits += h[i];
    }
    
    return hits;
}

In [ ]:
%%writefile bindings/cffi/exercises/shoot.h

void shoot(double *x, double *y, int *h, int n);

In [ ]:
%%writefile bindings/cffi/exercises/count.h

int count(int *h, int n);

In [ ]:
%%writefile bindings/cffi/exercises/piapprox.py

from cffi import FFI
ffibuilder = FFI()

ffibuilder.cdef("""void shoot(double *x, double *y, int *h, int n);
                   int count(int *h, int n);""")

ffibuilder.set_source("_pi_approx",
"""
    #include "shoot.h"
    #include "count.h"
""",
    sources=['shoot.c', 'count.c'],
    libraries=['m']) 

if __name__ == "__main__":
    ffibuilder.compile(verbose=True)

In [ ]:
! cd bindings/cffi/exercises/; python3 piapprox.py

In [ ]:
%%writefile bindings/cffi/exercises/main.py

import numpy as np
from _pi_approx import ffi, lib


n = 1_000_000
x = np.random.rand(n)
y = np.random.rand(n)
h = np.zeros(n)

xptr = ffi.cast("double *", ffi.from_buffer(x))
yptr = ffi.cast("double *", ffi.from_buffer(y))
hptr = ffi.cast("int *", ffi.from_buffer(h))
lib.shoot(xptr, yptr, hptr, n)
pi = 4*lib.count(hptr, n)/n

print(pi)

## Cython - Exercise

The way CFFI treats your code does not suite you, therefore, you want to give Cython a chance. Implement the necessary parts now for Cython.

___Remarks:___ Again you do not need to write any C code. Only the 5th, 6th, and 7th cells below require some Python code.

In [ ]:
%%writefile bindings/cython/exercises/shoot.c

# include <math.h>


void shoot(double *x, double *y, int *h, int n) {
    for (int i = 0; i < n; i++) {
            h[i] = (sqrt(x[i]*x[i] + y[i]*y[i]) <= 1.0) ? 1 : 0;
    }
}

In [ ]:
%%writefile bindings/cython/exercises/count.c

int count(int *h, int n) {
    int hits = 0;
    
    for (int i = 0; i < n; i++) {
        hits += h[i];
    }
    
    return hits;
}

In [ ]:
%%writefile bindings/cython/exercises/shoot.h

void shoot(double *x, double *y, int *h, int n);

In [ ]:
%%writefile bindings/cython/exercises/count.h

int count(int *h, int n);

In [ ]:
%%writefile bindings/cython/exercises/c_shoot.pxd


    void shoot(double *x, double *y, int *h, int n)

In [ ]:
%%writefile bindings/cython/exercises/c_count.pxd


    int count(int *h, int n)

In [ ]:
%%writefile bindings/cython/exercises/py_piapprox.pyx

from c_shoot cimport shoot
from c_count cimport count

cimport numpy as np
np.import_array()


def piapprox_py():
    return None

In [ ]:
%%writefile bindings/cython/exercises/setup.py

import numpy as np
from distutils.core import setup
from distutils.extension import Extension
from Cython.Build import cythonize


sourcefiles = ['py_piapprox.pyx', 'shoot.c', 'count.c']
extensions = [Extension("piapprox", sourcefiles, include_dirs=[np.get_include()])]

setup(
    ext_modules=cythonize(extensions),
    include_dirs=[np.get_include()]
)

In [ ]:
! cd bindings/cython/exercises ; python3 setup.py build_ext --inplace

In [ ]:
%%writefile bindings/cython/exercises/main.py

import numpy as np
from piapprox import piapprox_py


n = 1_000_000
x = np.random.rand(n)
y = np.random.rand(n)
h = np.zeros(n, dtype=np.intc)
pi = 4*piapprox_py(x, y, h)/n

print(pi)

In [ ]:
! cd bindings/cython/exercises ; python3 main.py

#### Cython - Solution Proposal

In [ ]:
%%writefile bindings/cython/exercises/shoot.c

# include <math.h>


void shoot(double *x, double *y, int *h, int n) {
    for (int i = 0; i < n; i++) {
            h[i] = (sqrt(x[i]*x[i] + y[i]*y[i]) <= 1.0) ? 1 : 0;
    }
}

In [ ]:
%%writefile bindings/cython/exercises/count.c

int count(int *h, int n) {
    int hits = 0;
    
    for (int i = 0; i < n; i++) {
        hits += h[i];
    }
    
    return hits;
}

In [ ]:
%%writefile bindings/cython/exercises/shoot.h

void shoot(double *x, double *y, int *h, int n);

In [ ]:
%%writefile bindings/cython/exercises/count.h

int count(int *h, int n);

In [ ]:
%%writefile bindings/cython/exercises/c_shoot.pxd

cdef extern from "shoot.h":
    void shoot(double *x, double *y, int *h, int n)

In [ ]:
%%writefile bindings/cython/exercises/c_count.pxd

cdef extern from "count.h":
    int count(int *h, int n)

In [ ]:
%%writefile bindings/cython/exercises/py_piapprox.pyx

from c_shoot cimport shoot
from c_count cimport count

cimport numpy as np
np.import_array()


def piapprox_py(np.ndarray[double, ndim=1, mode="c"] x not None,
                np.ndarray[double, ndim=1, mode="c"] y not None,
                np.ndarray[int, ndim=1, mode="c"] h not None):
    shoot(&x[0], &y[0], &h[0], x.shape[0])
    return count(&h[0], x.shape[0])

In [ ]:
%%writefile bindings/cython/exercises/setup.py

import numpy as np
from distutils.core import setup
from distutils.extension import Extension
from Cython.Build import cythonize


sourcefiles = ['py_piapprox.pyx', 'shoot.c', 'count.c']
extensions = [Extension("piapprox", sourcefiles, include_dirs=[np.get_include()])]

setup(
    ext_modules=cythonize(extensions),
    include_dirs=[np.get_include()]
)

In [ ]:
! cd bindings/cython/exercises ; python3 setup.py build_ext --inplace

In [ ]:
%%writefile bindings/cython/exercises/main.py

import numpy as np
from piapprox import piapprox_py


n = 1_000_000
x = np.random.rand(n)
y = np.random.rand(n)
h = np.zeros(n, dtype=np.intc)
pi = 4*piapprox_py(x, y, h)/n

print(pi)

In [ ]:
! cd bindings/cython/exercises ; python3 main.py

# f2py - Exercise

As a relaxation exercise, generate wrappers with `f2py` for the Fortran file below and call them.

___Remarks:___ It is not necessary to write any Fortran code. Only the 2nd and 3rd cells below require adaptation.

In [ ]:
%%writefile bindings/f2py/exercises/add.f90

subroutine add(a, b, c, n)

 implicit none

 real(kind=8), intent(in) :: a(n)
 real(kind=8), intent(in) :: b(n)
 real(kind=8), intent(out) :: c(n)
 integer :: n

 c = a + b

end subroutine

In [ ]:
! cd bindings/f2py/exercises ; f2py3 ...

In [ ]:
%%writefile bindings/f2py/exercises/addpy.py

import numpy as np
from fmadd import add

x = np.arange(10.)
y = np.arange(10.)
z = None
print(z)

In [ ]:
! python3 bindings/f2py/fibpy.py

#### f2py - Solution Proposal

In [ ]:
%%writefile bindings/f2py/exercises/add.f90

subroutine add(a, b, c, n)

 implicit none

 real(kind=8), intent(in) :: a(n)
 real(kind=8), intent(in) :: b(n)
 real(kind=8), intent(out) :: c(n)
 integer :: n

 c = a + b

end subroutine

In [ ]:
! cd bindings/f2py/exercises ; f2py3 -c add.f90 -m fmadd

In [ ]:
%%writefile bindings/f2py/exercises/addpy.py

import numpy as np
from fmadd import add

x = np.arange(10.)
y = np.arange(10.)
z = add(x, y)
print(z)

In [ ]:
! python3 bindings/f2py/fibpy.py